# Target Visibility

In [ ]:
# Parameters cell. Set defaults here
import datetime
ra = 270.
dec = -20.
name = 'Test Target'
time = datetime.datetime.fromisoformat('2025-05-01T23:59:00Z')

In [ ]:
import numpy as np
import scipy

from astropy.coordinates import SkyCoord, EarthLocation, get_body
import astropy.units as u
from astropy.time import Time
from astroplan import Observer, FixedTarget
from astroplan.plots import plot_airmass, plot_sky

import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
observer = Observer.at_site('LSST')

In [ ]:
time = Time(time)
coord = SkyCoord(ra=ra, dec=dec, unit='deg', frame='icrs')
target = FixedTarget(coord, name=name)

In [ ]:
def rotatorSpeed(lat, alt, az):
    """
    All inputs in decimal degrees
    """
    return ((360 * u.deg) / (86400 * u.s)) * np.cos(np.radians(lat)) * np.cos(np.radians(az)) / np.cos(np.radians(alt))

## Basic Information

In [ ]:
time_midnight = observer.midnight(time, which='nearest')
time_sunset = observer.sun_set_time(time_midnight, which='previous')
time_sunrise = observer.sun_rise_time(time_midnight, which='next')

if observer.moon_altaz(time_midnight).alt.value > 0.:
    time_moonrise = observer.moon_rise_time(time_midnight, which='previous')
    time_moonset = observer.moon_set_time(time_midnight, which='next')
else:
    time_moonrise = observer.moon_rise_time(time_midnight, which='next')
    time_moonset = observer.moon_set_time(time_midnight, which='previous')

location = EarthLocation.of_site('lsst')
sun = get_body('sun', time, location)
ra_opposition = (sun.ra.deg - 180.) % 360.
moon = get_body('moon', time, location)

print('Time (UTC) =', time)
print('Time Evening Civil Twilight (UTC) =', observer.twilight_evening_civil(time_midnight, which='previous').iso)
print('Time Evening Nautical Twilight (UTC) =', observer.twilight_evening_nautical(time_midnight, which='previous').iso)
print('Time Evening Astronomical Twilight (UTC) =', observer.twilight_evening_astronomical(time_midnight, which='previous').iso)
print('Time Midnight (UTC) =', time_midnight.iso)
print('Time Morning Astronomical Twilight (UTC) =', observer.twilight_morning_astronomical(time_midnight, which='next').iso)
print('Time Morning Nautical Twilight (UTC) =', observer.twilight_morning_nautical(time_midnight, which='previous').iso)
print('Time Morning Civil Twilight (UTC) =', observer.twilight_morning_civil(time_midnight, which='previous').iso)
print('Local Sidereal Time (deg) = %.3f'%(observer.local_sidereal_time(time).deg))
print('Solar Opposition RA (deg) = %.3f'%(ra_opposition))
print('\nTarget = %s'%(name))
print('(RA, Dec) = (%.2f, %2f)'%(ra, dec))
print('RA (decimal hours) = %.4f'%(coord.icrs.ra.hour))
print('(alt, az) = (%.2f, %.2f)'%(observer.altaz(time, coord).alt.deg, observer.altaz(time, coord).az.deg))
print('Target Transit (UTC) =', observer.target_meridian_transit_time(time_midnight, coord, which='nearest').iso)
print('\nMoon Illumination = %.3f'%(observer.moon_illumination(time)))
print('Moonrise time (UTC) =', time_moonrise.iso)
print('Moonset time (UTC) =', time_moonset.iso)
print('Moon (RA, Dec) = (%.2f, %.2f)'%(moon.ra.deg, moon.dec.deg))
print('Moon (alt, az) = (%.2f, %.2f)'%(observer.altaz(time, moon).alt.deg, observer.altaz(time, moon).az.deg))
print('Lunar Separation (deg) =', observer.moon_altaz(time).separation(observer.altaz(time, coord)).deg)

In [ ]:
time_array = time_sunset + np.arange(0., (time_sunrise - time_sunset).to(u.hr).value, 0.1) * u.hr

## Visibility on Specified Night

In [ ]:
def plotMoon(time):
    time_sun_set = observer.sun_set_time(time, which='previous')
    time_sun_rise = observer.sun_rise_time(time, which='next')
    if observer.moon_altaz(time_sun_set).alt.value > 0.:
        print('Moon is up at start of night')
        time_moonset = observer.moon_set_time(time_sun_set, which='next')
        plt.axvspan(
            time_sun_set.plot_date, 
            min(time_moonset, time_sun_rise).plot_date, 
            color='yellow', alpha=0.2, zorder=-999,
        )
    if observer.moon_altaz(time_sun_rise).alt.value > 0.:
        print('Moon is up at end of night')
        time_moonrise = observer.moon_rise_time(time_sun_rise, which='previous')
        plt.axvspan(
            max(time_moonrise, time_sun_set).plot_date,
            time_sun_rise.plot_date,
            color='yellow', alpha=0.2, zorder=-999,
        )

In [ ]:
plt.figure()
plt.clf()
ax = plot_airmass(target, observer, time, brightness_shading=True, altitude_yaxis=True)
plotMoon(time)
ax.legend(loc='lower right', title='Moon Illumination = %.2f'%(observer.moon_illumination(time)))
plt.show()

In [ ]:
plt.clf()
plot_sky(target, observer, time_array)
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

In [ ]:
plt.figure()
plot_sky(target, observer, time)

moon_style_kwargs = {
    'edgecolor': 'black', 
    'facecolor': 'white',
    's': 100,
}
if observer.moon_altaz(time).alt.value > 0.:
    plot_sky(
        FixedTarget(
            SkyCoord(moon.ra.deg, moon.dec.deg, unit='deg', frame='icrs'), 
            name='Moon Illumination = %.2f'%(observer.moon_illumination(time))
        ), 
        observer, 
        time, 
        style_kwargs=moon_style_kwargs
    )
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.suptitle(time)
plt.show()

In [ ]:
alt_array = observer.altaz(time_array, coord).alt.deg
az_array = observer.altaz(time_array, coord).az.deg
rotator_speed_array = rotatorSpeed(observer.latitude.deg, alt_array, az_array).value

print('%30s%15s%15s%30s'%('Time (UTC)', 'Alt (deg)', 'Az (deg)', 'Rotator Speed (deg/sec)'))
for ii in range(0, len(time_array)):
    print('%30s%15.2f%15.2f%30.3f'%(time_array[ii].iso, alt_array[ii], az_array[ii], rotator_speed_array[ii]))

# Observation Windows

In [ ]:
def printTime(time):
    return ':'.join(time.iso.split()[1].split(':')[0:2])

In [ ]:
time_midnight = observer.midnight(time, which='nearest')
time_sunset = observer.sun_set_time(time_midnight, which='previous')
time_sunrise = observer.sun_rise_time(time_midnight, which='next')
time_evening_astronomical_twilight = observer.twilight_evening_astronomical(time_midnight, which='previous')
time_morning_astronomical_twilight = observer.twilight_morning_astronomical(time_midnight, which='next')

time_array = time_evening_astronomical_twilight \
    + np.arange(0., (time_morning_astronomical_twilight - time_evening_astronomical_twilight).to(u.hr).value, 0.1) * u.hr


alt_min = 45.
alt_max = 83.
lunar_separation_min = 30.
close_hrs_sunset = 2.
az_hrs_sunset = 3.
az_min = 180.
az_max = 270.
print('alt_min (deg) = %.3f'%(alt_min))
print('alt_max (deg) = %.3f'%(alt_max))
print('lunar_separation_min (deg) = %.3f'%(lunar_separation_min))
print('close_hrs_sunset (hrs) = %.3f'%(close_hrs_sunset))
print('az_hrs_sunset (hrs) = %.3f'%(close_hrs_sunset))
print('az_min (deg) = %.3f'%(az_min))
print('az_max (deg) = %.3f'%(az_max))

print('\nday_obs = %s\n'%(time_sunset.iso.split()[0]))

for target in [target]:
    time_transit = observer.target_meridian_transit_time(time_midnight, target.coord, which='nearest')
    print(target.name)
    print('  Transit\t%s (UTC)'%(printTime(time_transit)))

    alt_array = observer.altaz(time_array, target.coord).alt.deg
    az_array = observer.altaz(time_array, target.coord).az.deg

    altitude_acceptable = (
        (alt_array > alt_min) \
        & (alt_array < alt_max)
    )
    twilight_acceptable = (
        (observer.sun_altaz(time_array).alt.value < -18.) \
        & ((time_array - time_sunrise).to(u.hour) < (-1. * close_hrs_sunset * u.hour))
    )
    azimuth_acceptable = (
        ((time_array - time_sunrise).to(u.hour) < (-1. * az_hrs_sunset * u.hour)) \
        | ((az_array > az_min) & (az_array < az_max))
    )
    lunar_separation_acceptable = (
        observer.moon_altaz(time).separation(observer.altaz(time, target.coord)).deg > lunar_separation_min
    )

    criteria = altitude_acceptable \
        &  twilight_acceptable \
        & azimuth_acceptable \
        & lunar_separation_acceptable
    
    observable_time_ranges = []
    labels, n_labels = scipy.ndimage.label(criteria)
    for label in range(0, n_labels):
        selection = np.where(labels == (label + 1))[0]

        time_start = time_array[np.min(selection)]
        time_end = time_array[np.max(selection)]
    
        print('  Observable\t%s - %s (UTC)'%(
            printTime(time_start),
            printTime(time_end),
        ))
    print('')

## Visibility over the Next Month

In [ ]:
time_plus_month = time + 30 * u.day

plt.figure()
plt.clf()
ax = plot_airmass(target, observer, time_plus_month, brightness_shading=True, altitude_yaxis=True)
plotMoon(time_plus_month)
ax.legend(loc='lower right', title='Moon Illumination = %.2f'%(observer.moon_illumination(time_plus_month)))
plt.show()

In [ ]:
time_next_month = time + np.arange(0., 30., 1.,) * u.day

lunar_illumination_next_month = observer.moon_illumination(time_next_month)

In [ ]:
x_ticks = Time(np.floor(time_next_month.mjd), format='mjd')
x_tick_labels = np.array([_.iso.split()[0] for _ in x_ticks])

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.axhspan(0., 20., alpha=0.2, color='black')

#for target in targets:
lunar_separation_next_month = observer.moon_altaz(time_next_month).separation(observer.altaz(time_next_month, target.coord)).deg
ax1.plot(time_next_month.mjd, lunar_separation_next_month, label=target.name)

ax1.set_ylabel('Lunar Separation (deg)')
ax1.set_xticks(x_ticks.mjd[::3], x_tick_labels[::3], rotation=45.)
ax1.set_ylim(0., 180.)

ax2.plot(time_next_month.mjd, lunar_illumination_next_month, c='black', ls='--')
ax2.set_ylabel('Lunar Illumination')
ax2.set_ylim(0.,1.)

ax1.legend(loc='upper center')

fig.tight_layout()